In [24]:
# !pip3 install datasets
# !pip3 install torch

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence

In [25]:
from gensim.models import KeyedVectors

#TODO: replace the path with your embedding model on your computer 
word2vec_model_path = '~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)


In [27]:
from datasets import load_dataset
dataset = load_dataset("imdb", split='train')

#Tokenizer
tokenizer = get_tokenizer('basic_english')

# Build vocabulary
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(dataset['text']), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [58]:
dataset.shape

(25000, 2)

In [60]:
dataset[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [28]:
embedding_dim = 300  # Word2Vec vector size
weight_matrix = torch.zeros(len(vocab), embedding_dim)

for word, index in vocab.get_stoi().items():
    try:
        word_vector = word2vec_model[word]
        weight_matrix[index] = torch.from_numpy(word_vector).clone()
    except KeyError:
        # For words not in Word2Vec, initialize randomly
        weight_matrix[index] = torch.randn(embedding_dim)


In [29]:
# Build the Sentiment Model
class SentimentModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, vocab_size, weight_matrix):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.load_state_dict({'weight': weight_matrix})
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text):
        embedded = self.embedding(text)
        _, (hidden, _) = self.lstm(embedded)
        return self.fc(hidden.squeeze(0))

# Prepare DataLoader
def collate_batch(batch):
    label_list, text_list = [], []
    for _text, _label in batch:
        processed_text = [vocab[token] for token in tokenizer(_text)]
        label_list.append(_label)
        text_list.append(torch.tensor(processed_text, dtype=torch.int64))
    return pad_sequence(text_list, padding_value=vocab["<pad>"]), torch.tensor(label_list, dtype=torch.float32)

train_dataset = list(zip(dataset['text'], dataset['label']))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_batch)

model = SentimentModel(embedding_dim, 256, 1, len(vocab), weight_matrix)

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss() #loss function


In [30]:
num_epochs = 3

for epoch in range(num_epochs):
    total_loss = 0
    for texts, labels in train_loader:
        optimizer.zero_grad()
        predictions = model(texts).squeeze(1)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(train_loader)}")


Epoch 1, Average Loss: 0.6925777795410156
Epoch 2, Average Loss: 0.6794356470489502
Epoch 3, Average Loss: 0.39065737891197205


In [45]:
def predict_sentiment(model, sentence):
    model.eval() 
    tokens = [vocab[token] for token in tokenizer(sentence)]
    padded_tokens = pad_sequence([torch.tensor(tokens, dtype=torch.int64)], padding_value=vocab["<pad>"])
    with torch.no_grad():
        prediction = torch.sigmoid(model(padded_tokens))
    return prediction.item()


Sentiment Analysis: Positive


In [54]:
# Example usage
sentence = "This is the best movie!"
prediction = predict_sentiment(model, sentence)
print('Positive' if prediction > 0.5 else 'Negative')
print(prediction)

Positive
0.8908994197845459


In [62]:
# Example usage
sentence = "Worst thing I've watched"
prediction = predict_sentiment(model, sentence)
print('Positive' if prediction > 0.5 else 'Negative')
print(prediction)

Negative
0.036396417766809464


In [57]:
# Example usage
sentence = "I don't think this is a good movie"
prediction = predict_sentiment(model, sentence)
print('Positive' if prediction > 0.5 else 'Negative')
print(prediction)

Positive
0.7404760718345642
